One of the simplest applications of metaclasses is verifying that a class was defined
correctly. When you’re building a complex class hierarchy, you may want to enforce style,
require overriding methods, or have strict relationships between class attributes.
Metaclasses enable these use cases by providing a reliable way to run your validation code
each time a new subclass is defined.

Often a class’s validation code runs in the __init__ method, when an object of the
class’s type is constructed (see Item 28: “Inherit from collections.abc for Custom
Container Types” for an example). Using metaclasses for validation can raise errors much
earlier.

Before I get into how to define a metaclass for validating subclasses, it’s important to
understand the metaclass action for standard objects. A metaclass is defined by inheriting
from type. In the default case, a metaclass receives the contents of associated class
statements in its __new__ method. Here, you can modify the class information before the
type is actually constructed:

The metaclass has access to the name of the class, the parent classes it inherits from, and
all of the class attributes that were defined in the class’s body.


In [3]:
import logging
from pprint import pprint
from sys import stdout as STDOUT


# Example 1
class Meta(type):
    def __new__(meta, name, bases, class_dict):
        orig_print = __builtins__.print
        print = pprint
        print((meta, name, bases, class_dict))
        print = orig_print
        return type.__new__(meta, name, bases, class_dict)

class MyClass(object, metaclass=Meta):
    stuff = 123

    def foo(self):
        pass

(<class '__main__.Meta'>,
 'MyClass',
 (<class 'object'>,),
 {'__module__': '__main__',
  '__qualname__': 'MyClass',
  'foo': <function MyClass.foo at 0x1040aa620>,
  'stuff': 123})


You can add functionality to the Meta.__new__ method in order to validate all of the
parameters of a class before it’s defined. For example, say you want to represent any type
of multisided polygon. You can do this by defining a special validating metaclass and
using it in the base class of your polygon class hierarchy. Note that it’s important not to
apply the same validation to the base class.



In [4]:
# Example 3
class ValidatePolygon(type):
    def __new__(meta, name, bases, class_dict):
        # Don't validate the abstract Polygon class
        if bases != (object,):
            if class_dict['sides'] < 3:
                raise ValueError('Polygons need 3+ sides')
        return type.__new__(meta, name, bases, class_dict)

class Polygon(object, metaclass=ValidatePolygon):
    sides = None  # Specified by subclasses

    @classmethod
    def interior_angles(cls):
        return (cls.sides - 2) * 180

class Triangle(Polygon):
    sides = 3

print(Triangle.interior_angles())

180


If you try to define a polygon with fewer than three sides, the validation will cause the
class statement to fail immediately after the class statement body. This means your
program will not even be able to start running when you define such a class.


In [5]:
# Example 4
try:
    print('Before class')
    class Line(Polygon):
        print('Before sides')
        sides = 1
        print('After sides')
    print('After class')
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-5-c2f600ebbf79>", line 4, in <module>
    class Line(Polygon):
  File "<ipython-input-4-33380c00d8d2>", line 7, in __new__
    raise ValueError('Polygons need 3+ sides')
ValueError: Polygons need 3+ sides


Before class
Before sides
After sides


* Use metaclasses to ensure that subclasses are well formed at the time they are defined, before objects of their type are constructed.
* Metaclasses have slightly different syntax in Python 2 vs. Python 3. 
* The __new__ method of metaclasses is run after the class statement’s entire body has been processed.